<a href="https://colab.research.google.com/github/kperv/summarizer_app/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summarization project

for ods.ai *Natural Language Processing course*


## Clustering sentence embeddings for text summarization


*   Text is broken into sentences by Spacy
*   Sentences are tokenized by pretrained BertTokenizer
*   Word embeddings are collected brom Bert and averaged to get sentence embeddings
*   sklearn is used for KMeans Clustering
*   Evaluation metric is rouge


## Installations and imports

In [43]:
%%capture
!pip install transformers
!pip install datasets
!pip install https://huggingface.co/spacy/ru_core_news_md/resolve/main/ru_core_news_md-any-py3-none-any.whl
!pip install https://huggingface.co/spacy/es_core_news_md/resolve/main/es_core_news_md-any-py3-none-any.whl
!git clone https://github.com/kperv/summarizer_app.git
!pip install -r summarizer_app/requirements.txt

In [44]:
import os
import numpy as np
import spacy
import nltk
import transformers
import datasets
import sklearn
import pandas as pd

from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
from transformers import BertTokenizer, BertModel
from rouge import Rouge

In [45]:
nlp = spacy.load("ru_core_news_md")

### requirements

In [46]:
print("numpy=={}".format(np.__version__))
print("spacy=={}".format(spacy.__version__))
print("transformers=={}".format(transformers.__version__))
print("sklearn=={}".format(sklearn.__version__))

numpy==1.19.5
spacy==3.2.1
transformers==4.13.0
sklearn==1.0.1


## Extractive text summarization

In [47]:
number = 3
lang='ru'

Some random paragraph from news

In [48]:
text = "Очень непросто делать прогнозы. С одной стороны, достаточно долго длится фаза подъема, после 11 июня началась третья волна в Свердловской области. Мы достигли очень высокого уровня заболеваемости и смертности и стабилизировались на нем. Как долго это будет продолжаться, зависит от доли восприимчивого к вирусу населения. На момент начала третьей волны число привитых или переболевших свердловчан не превышало 50 процентов, — отметил Соловьев. — Чтобы третья волна остановилась, мы должны достичь высокого уровня коллективного иммунитета. С помощью вакцинации мы уже не успеваем его достичь. Второй вариант — заболеваемость. Официальная статистика не отражает реальное число людей, которые встретились с коронавирусом. Сейчас приблизительно 65% жителей переболели или вакцинировались от коронавируса. Это должно сказываться на снижении заболеваемости, но пока этого не происходит."

### Separate sentences with SpaCy

In [49]:
def break_text(text):
  doc = nlp(text)
  assert doc.has_annotation("SENT_START")
  sentences = [str(sent) for sent in doc.sents]
  return sentences

In [50]:
sentences = break_text(text)

### Convert words to tokens and run through Bert encoder to get word embeddings

In [51]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained("bert-base-multilingual-cased")
encoded_sentences = tokenizer(sentences, truncation=True, padding=True, return_tensors="pt")
outputs = model(encoded_sentences.input_ids)
embeddings = outputs.last_hidden_state

I'm using *mean* to get sentence embeddings. This step can be improved.

In [52]:
embeddings = embeddings.detach().numpy()
embeddings = embeddings.mean(axis=1)

Cluster embeddings in a space into n (number) clusters to get centroids. Take distance metric between sentence embeddings and centroids and choose n closest to put into the summary.

### Collect summary sentences together

In [53]:
kmeans = KMeans(n_clusters=number).fit(embeddings)
centroids = kmeans.cluster_centers_
result = pairwise_distances_argmin_min(embeddings, centroids)
summary_sent_positions = list(np.argsort(result[1])[:number])

In [54]:
summary = ""
for idx in summary_sent_positions:
  summary += sentences[idx]
summary

'На момент начала третьей волны число привитых или переболевших свердловчан не превышало 50 процентов, — отметил Соловьев.Очень непросто делать прогнозы.Как долго это будет продолжаться, зависит от доли восприимчивого к вирусу населения.'

## Evaluate solution on 200 samples

In [55]:
def collect_modified_dataset(data_path):
    transformed_df = pd.DataFrame()
    for dataset_part in os.listdir(data_path):
        part_path = os.path.join(data_path, dataset_part)
        df_slice = pd.read_csv(part_path)
        transformed_df = pd.concat([transformed_df, df_slice])
    return transformed_df

In [56]:
def get_rouge_score(sample):
    rouge = Rouge()
    preprocess_exs = lambda exs : [ex.strip().lower() for ex in exs]
    predictions = []
    predictions.append(sample['summary'])
    predictions = preprocess_exs(predictions)
    references = []
    references.append(sample.text)
    references = preprocess_exs(references)
    predictions = [pred if len(pred) else 'а' for pred in predictions]
    rouge_scores =  rouge.get_scores(predictions, references, avg=True)
    return {k: v['f'] * 100 for k, v in rouge_scores.items()}

In [57]:
def add_metrics(dataset):
    dataset = dataset.loc[:, ['text', 'summary']]
    dataset[['rouge-1', 'rouge-2', 'rouge-l']] = 0, 0, 0
    df = pd.DataFrame(list(dataset.apply(get_rouge_score, axis=1).values))
    dataset = df.combine_first(dataset)
    dataset = dataset.reindex(
        columns=['text', 'summary', 'rouge-1', 'rouge-2', 'rouge-l']
    )
    return dataset

In [58]:
data_dir = os.getcwd()
data_path = data_dir + "/summarizer_app/data"
df = collect_modified_dataset(data_path)
df = add_metrics(df)

In [59]:
round(df['rouge-1'].mean(), 3)

35.922

In [60]:
round(df['rouge-2'].mean(), 3)

31.15

In [61]:
round(df['rouge-l'].mean(), 3)

26.701